In [ ]:
import numpy as np
import matplotlib.colors as colors
from scipy.interpolate import griddata, NearestNDInterpolator
import matplotlib.pyplot as pl
from numpy import uint32, uint64, float64, float32, linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos
import matplotlib.colors as mc
###################################
# # XYZ are cartesian coordinates centered at the Sun position. X axis points to the GC, Z axis up to north Galactic pole.
# # xyz are cartesian coordinates centered at the GC with same orientations as XYZ.
# # lbr are spherical coordinates centered at the Sun position (i.e., the usual Galactic coordinates)
# # the Sun position is assumed to be (xsun,ysun) = (-R0,0)
# # the Sun velocity is assumed to be vsun pointing in the pointing in the positive y direction
# # the following functions convert back and forth between the above coordinates
###################################


header_names = ('num_particles', 'mass', 'time', 'redshift', 'flag_sfr', 'flag_feedback', 'num_particles_total', 'flag_cooling', \
                'num_files', 'boxsize', 'omega0', 'omegaLambda', 'hubble0', 'flag_stellarage', 'flag_metals', \
                'npartTotalHighWord', 'flag_entropy_instead_u', 'flag_doubleprecision', \
                'flag_lpt_ics', 'lpt_scalingfactor', 'flag_tracer_field', 'composition_vector_length', 'buffer')

header_sizes = ((uint32, 6), (float64, 6), (float64, 1), (float64, 1), (uint32, 1), (uint32, 1), \
                (uint32, 6), (uint32, 1), (uint32, 1), (float64, 1), (float64, 1), (float64, 1), \
                (float64, 1), (uint32, 1), (uint32, 1), (uint32, 6), (uint32, 1), (uint32, 1), \
                (uint32, 1), (float32, 1), (uint32, 1), (uint32, 1), (np.uint8, 40))

def read_header(f):
  """ Read the binary header file into a dictionary """
  block_size = np.fromfile(f, uint32, 1)[0]
  header = dict(((name, np.fromfile(f, dtype=size[0], count=size[1])) \
               for name, size in zip(header_names, header_sizes)))
  assert(np.fromfile(f, uint32, 1)[0] == 256)
  return header

def readu(f, dtype=None, components=1):
  """ Read a numpy array from the unformatted fortran file f """
  data_size = np.fromfile(f, uint32, 1)[0]
  count = data_size/np.dtype(dtype).itemsize
  arr = np.fromfile(f, dtype, count)
  final_block = np.fromfile(f, uint32, 1)[0]
  # check the flag at the beginning corresponds to that at the end
  assert(data_size == final_block)
  return arr

def readIDs(f, count):
  """ Read a the ID block from a binary snapshot file """
  data_size = np.fromfile(f, uint32, 1)[0]
  f.seek(-4, 1)
  count = int(count)
  if data_size / 4 == count: dtype = uint32
  elif data_size / 8 == count: dtype = uint64
  else: raise Exception('Incorrect number of IDs requested')
  print "ID type: ", dtype
  return readu(f, dtype, 1)

def read_snapshot_file(filename):
  """ Reads a binary arepo file """
  f = open(filename, mode='rb')
  print "Loading file %s" % filename
  data = {} # dictionary to hold data
  # read the header
  header = read_header(f)
  nparts = header['num_particles']
  masses = header['mass']
  total = nparts.sum()
  n_gas = nparts[0]
  print 'Particles', nparts
  print 'Gas particles', n_gas
  print 'Time = ', header['time']
  precision = float32
  print 'Reading positions'
  data['pos'] = readu(f, precision, 3).reshape((total, 3))
  print 'Reading velocities'
  data['vel'] = readu(f, precision, 3).reshape((total, 3))
  print 'Reading IDs'
  data['id'] = readIDs(f, total)
  print 'Reading masses'
  data['mass'] = readu(f, precision, 1)
  print 'Reading internal energy'
  data['u_therm'] = readu(f, precision, 1)
  print 'Reading densities'
  data['rho'] = readu(f, precision, 1)
  #print 'Reading chemical abundances'
  #data['chem'] = readu(f, precision, 3).reshape((n_gas, 3))
  #print 'Reading dust temperatures'
  #data['tdust'] = readu(f, precision, 1)
  f.close()
  return data, header

def XYZ2lbr(X,Y,Z):
  r = sqrt(X**2+Y**2+Z**2)
  l = arctan2(Y,X)
  b = pi/2 - arccos(Z/r)
  return l,b,r

def lbr2XYZ(l,b,r):
  X = r*sin(b+pi/2)*cos(l)
  Y = r*sin(b+pi/2)*sin(l)
  Z = r*cos(b+pi/2)
  return X,Y,Z

def xyz2XYZ(x,y,z,R0=80.0):
  return x+R0,y,z

def XYZ2xyz(X,Y,Z,R0=80.0):
  return X-R0,Y,Z

def xyz2lbr(x,y,z):
  X,Y,Z = xyz2XYZ(x,y,z)
  l,b,r = XYZ2lbr(X,Y,Z)
  return l,b,r

def lbr2xyz(l,b,r):
  X,Y,Z = lbr2XYZ(l,b,r)
  x,y,z = XYZ2xyz(X,Y,Z)
  return x,y,z

def vxyz2vlbr(x,y,z,vx,vy,vz,vsun=220.0):
  # see wiki "vector fields in spherical coords" for formulas
  X,Y,Z = xyz2XYZ(x,y,z)
  l,b,r = XYZ2lbr(X,Y,Z)
  rhat = [sin(b+pi/2)*cos(l),sin(b+pi/2)*sin(l),cos(b+pi/2)]
  bhat = [cos(b+pi/2)*cos(l),cos(b+pi/2)*sin(l),-sin(b+pi/2)] 
  lhat = [-sin(l),cos(l),0]
  vr = vx*rhat[0] + (vy-vsun)*rhat[1] + vz*rhat[2]
  vb = vx*bhat[0] + (vy-vsun)*bhat[1] + vz*bhat[2]
  vl = vx*lhat[0] + (vy-vsun)*lhat[1]
  return l,b,r,vl,vb,vr

#############################
# Simple function to rotate in a plane
#############################

def rotate(x,y,theta):
  xprime = x*cos(theta) - y*sin(theta)
  yprime = x*sin(theta) + y*cos(theta)
  return xprime, yprime

#internal arepo units in cgs
arepoLength = 3.0856e20
arepoMass = 1.991e33
arepoVel = 1.0e5
arepoTime = arepoLength/arepoVel
arepoDensity = arepoMass/arepoLength/arepoLength/arepoLength
arepoEnergy= arepoMass*arepoVel*arepoVel
arepoColumnDensity = arepoMass/arepoLength/arepoLength

In [ ]:
# Set paths
out_path = '/home/hph16101/arepo_sims_hpc/sim_2D_test/OUTPUT/'
save_path = '/home/hph16101/arepo_sims_hpc/sim_2D_test/anal/'
data, header = read_snapshot_file(out_path+'whole_disk_200')
t = header['time']
omega = 4.0
x,y,z = data['pos'].T
x,y,z = x-120, y-120, z-10
phi = np.radians(20)
x,y = rotate(x,y,omega*t - phi)
vx,vy,vz = data['vel'].T
vx,vy = rotate(vx,vy,omega*t - phi)
rho = data['rho'] * arepoDensity
mass = data['mass']
energy_per_unit_mass = data['u_therm']*arepoEnergy/arepoMass
#xH2, xHp, xCO = data['chem'].T
l,b,r,vl,vb,vr = vxyz2vlbr(x,y,z,vx,vy,vz)

In [ ]:
fig, ax = pl.subplots()
ax.hist2d(np.degrees(l),vr,bins=[600,600],weights=(mass/(r*r)),cmap='Greys',norm=colors.LogNorm())
ax.set_ylim([-250,250])
ax.set_xlim([90,-90])
ax.set_xlabel(v )
ax.set_ylabel()
pl.show()
fig.savefig(save_path+'lv.png')